# TFX on KubeFlow Pipelines Example

### Install TFX and KFP packages

In [ ]:
!pip3 install https://storage.googleapis.com/ml-pipeline/tfx/tfx-0.12.0rc0-py2.py3-none-any.whl 
!pip3 install https://storage.googleapis.com/ml-pipeline/release/0.1.10/kfp.tar.gz --upgrade


## Get the TFX repo with sample pipeline


In [ ]:
!git clone https://github.com/tensorflow/tfx

In [ ]:
# copy the trainer code to a storage bucket 
!gsutil cp tfx/examples/chicago_taxi_pipeline/taxi_utils.py gs://<my bucket>/

## Configure the TFX pipeline example

Reload this cell by running the load command to get the pipeline configuration file
```
%load tfx/examples/chicago_taxi_pipeline/taxi_pipeline_kubeflow.py
```

Configure:
- GCS storage bucket name (replace my-bucket)
- GCP project ID (replace my-gcp-project)
- Make sure the path to the taxi_utils.py is correct

The dataset in BigQuery has 100M rows, you can change the query parameters in WHERE clause to limit the number of rows used.


In [7]:
%load tfx/examples/chicago_taxi_pipeline/taxi_pipeline_kubeflow.py

## Compile the pipeline and submit a run to the Kubeflow cluster

In [ ]:
# Get or create a new experiment
import kfp
client = kfp.Client()
experiment = client.create_experiment("TFX Examples")
pipeline_filename = "chicago_taxi_pipeline_kubeflow.tar.gz"

#Submit a pipeline run
run_name = 'Run 1'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, {})
